# Set the namespaces based on _your_ deployment

In [1]:
# Paste the namespace of your baseline flow here. 
# You can find it in Metaflow GUI or in the command line print out when you do `python flow.py argo-workflows create`
champ_namespace = "production:mfprj-xsfdb3gtsiboqyrd-0-vqsy" # "production:mfprj-xsfdb3gtsiboqyrd-0-vqsy"
challenger_namespace = "production:mfprj-cfyzlfzievjlmsf4-0-tbgz" # "production:mfprj-cfyzlfzievjlmsf4-0-tbgz"
# champ_namespace = ... # 'production:mfprj-xsfdb3gtsiboqyrd-0-vqsy'
# challenger_namespace = ... # 'production:mfprj-cfyzlfzievjlmsf4-0-tbgz'

# Don't change this unless you know what you are doing and want to change the uvicorn command.
endpoint_uri_base = 'http://127.0.0.1:8000/'

In [2]:
from metaflow import Flow, namespace
FLOW_NAME = 'TitanicSurvivalPredictor'

namespace(challenger_namespace)
run = Flow(FLOW_NAME).latest_successful_run
model_type = run.data.model_type
print(model_type)

xgboost


In [3]:
run.data.model.predict()

/home/workspace/mambaforge/envs/full-stack-metaflow-corise/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


TypeError: XGBClassifier.predict() missing 1 required positional argument: 'X'

# Configure the endpoints to set model state on the server

In [4]:
use_challenger = False # If False, only load the champion model into memory on the server.

if use_challenger:
    slug = 'load-models?champ_namespace={}&challenger_namespace={}'.format(
        champ_namespace, challenger_namespace)
else:
    slug = 'load-models?champ_namespace={}'.format(champ_namespace)

load_models_endpoint = endpoint_uri_base + slug
print(load_models_endpoint)

http://127.0.0.1:8000/load-models?champ_namespace=production:mfprj-xsfdb3gtsiboqyrd-0-vqsy


# Run the uvicorn server

Open a terminal and run:
```sh
uvicorn model-server:api
```
This needs to stay running while you complete the remaining cells of the notebook.

# Load the champion, and maybe challenger, model states on the server

In [5]:
import requests

# A bit odd to make an API request to load models into memory, but isn't too different from what might happen in prod system.
# It works well in this POC to show the core functionality of an A/B test.
try: 
    response = requests.get(load_models_endpoint, verify=False, proxies={"https": endpoint_uri_base})
    print(response.json())
except requests.ConnectionError as e: 
    print("Did you run the server?!")

Running baseline model as champion.


# Get sample data

In [6]:
# get random data point
import pandas as pd
data = pd.read_csv("../data/titanic.csv")

inference_example = data.sample()
idx = inference_example.index[0]
true_target = inference_example['Survived'].values[0]

In [7]:
payload = inference_example.to_json()

# Request a prediction from server via basic A/B test

In [8]:
force_challenger_model = False # if False, 80% traffic to champ model, 20% traffic to challenger model.

pred_slug = 'get-pred?data={}'.format(payload)
if force_challenger_model:
    pred_slug += '&which_model=challenger'

In [1]:
prediction_endpoint = endpoint_uri_base + pred_slug
try:
    response = requests.get(prediction_endpoint, verify=False, proxies={"https": endpoint_uri_base})
    print(response.json())
except requests.ConnectionError as e: 
    print("Did you run the server?!")

NameError: name 'endpoint_uri_base' is not defined

# Verify A/B traffic is close to 80/20

In [10]:
counts = {"champ": 0, "challenger": 0}
for _ in range(100):
    response = requests.get(prediction_endpoint, verify=False, proxies={"https": endpoint_uri_base})
    counts[response.json()['model_used']] += 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [33]:
counts

{'champ': 4, 'challenger': 0}